## Wprowadzenie

W tym notatniku rozbudujemy podstawowy pipeline danych z Ćwiczenia 1 o zaawansowane techniki przetwarzania tekstu zgodnie z architekturą RAG:

1. Preprocessing danych
2. Techniki chunkowania
3. Generowanie embedingów
4. Indeksowanie i przygotowanie do wyszukiwania

Celem jest przygotowanie danych do efektywnego wykorzystania w modelu RAG (Retrieval-Augmented Generation).



## 1. Instalacja niezbędnych bibliotek

Do przetwarzania tekstu wykorzystamy popularne biblioteki jak LangChain, które ułatwiają pracę z danymi nieustrukturyzowanymi.



In [ ]:

# Instalacja niezbędnych bibliotek
%pip install langchain unstructured PyPDF2 sentence-transformers langchain-community markdown



In [ ]:
%pip install markdown


In [ ]:
# Import bibliotek
import pandas as pd
import numpy as np
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.document_loaders import PyPDFLoader, TextLoader, DirectoryLoader, UnstructuredMarkdownLoader
from langchain.embeddings import HuggingFaceEmbeddings
import re
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, FloatType

### 🔧 Ćwiczenie 1: Dodaj własny dokument Markdown
Załadowanie wcześniej przygotowanych danych

In [ ]:

# Odczyt danych ustrukturyzowanych z poprzedniego ćwiczenia
# Jeśli w Ćwiczeniu 1 zapisaliśmy dane do tabel Delta, możemy je odczytać:
try:
    df_structured = spark.table("customer_data")
    print(f"Odczytano dane ustrukturyzowane, liczba wierszy: {df_structured.count()}")
except:
    print("Brak tabeli customer_data - utwórz przykładowe dane")
    
    # Przykładowe dane jeśli tabela nie istnieje
    data = [("1", "Przewodnik użytkownika", "To jest przewodnik użytkownika opisujący funkcje produktu..."),
            ("2", "FAQ", "Najczęściej zadawane pytania dotyczące instalacji i konfiguracji..."),
            ("3", "Instrukcja techniczna", "Specyfikacja techniczna produktu zawierająca szczegółowe parametry...")]
    
    df_structured = spark.createDataFrame(data, ["id", "title", "content"])
    df_structured.write.format("delta").mode("overwrite").saveAsTable("customer_data")
    print(f"Utworzono przykładowe dane, liczba wierszy: {df_structured.count()}")


## 2. Odczyt danych nieustrukturyzowanych z poprzedniego ćwiczenia

Załóżmy, że w poprzednim ćwiczeniu połączyliśmy się z różnymi źródłami danych i zapisaliśmy je w formacie Delta Lake. Teraz odczytamy te dane i przygotujemy je do dalszego przetwarzania.



In [ ]:
# Ścieżka do Volume (uwaga: /Volumes/<catalog>/<schema>/<volume_name>/)
volume_path = "/Volumes/sqlday_edbworkshop/default/workshop_volume/docs/"

import os

# Upewnij się, że folder docelowy istnieje (możesz to pominąć – os.makedirs nie musi być konieczne dla Volume)
os.makedirs(volume_path, exist_ok=True)

sample_texts = [
    "# Dokumentacja produktu\n\nNasz produkt oferuje zaawansowane funkcje analityczne...\n\n## Instalacja\n\nAby zainstalować produkt, wykonaj następujące kroki...\n\n## Konfiguracja\n\nKonfiguracja produktu wymaga...",
    "# Przewodnik użytkownika\n\nW tym przewodniku opisujemy krok po kroku jak korzystać z...\n\n## Funkcje podstawowe\n\nPodstawowe funkcje obejmują...\n\n## Funkcje zaawansowane\n\nZaawansowane funkcje pozwalają na...",
    "# FAQ\n\n## Jak zresetować hasło?\n\nAby zresetować hasło, przejdź do ustawień...\n\n## Jak utworzyć nowy projekt?\n\nAby utworzyć nowy projekt, kliknij przycisk 'Nowy projekt'..."
]

# Zapis plików do Volume
for i, text in enumerate(sample_texts):
    with open(f"{volume_path}dokument_{i+1}.md", "w") as f:
        f.write(text)

print(f"✅ Utworzono przykładowe pliki w: {volume_path}")


In [ ]:

from langchain.document_loaders import DirectoryLoader, UnstructuredMarkdownLoader

try:
    loader = DirectoryLoader(
        path=volume_path,
        glob="**/*.md",
        loader_cls=UnstructuredMarkdownLoader
    )
    documents = loader.load()
    print(f"✅ Załadowano {len(documents)} dokumentów")
except Exception as e:
    print(f"❌ Błąd przy ładowaniu dokumentów: {e}")
    documents = []


## 3. Preprocessing danych

Przed chunkingiem należy przeprowadzić czyszczenie i wzbogacanie danych:
- Czyszczenie tekstu
- Dodanie metadanych



In [ ]:

# 3.1 Czyszczenie tekstu
def clean_text(text):
    """Funkcja czyszcząca tekst z niepotrzebnych znaków i formatowania"""
    # Usuwanie nadmiarowych białych znaków
    text = re.sub(r'\s+', ' ', text)
    # Usuwanie znaków specjalnych, które mogą zakłócać analizę
    text = re.sub(r'[^\w\s.,;:!?()-]', '', text)
    return text.strip()

# Czyszczenie tekstów w dokumentach
cleaned_documents = []
for doc in documents:
    cleaned_text = clean_text(doc.page_content)
    doc.page_content = cleaned_text
    cleaned_documents.append(doc)

# 3.2 Dodanie metadanych
for i, doc in enumerate(cleaned_documents):
    # Dodanie dodatkowych metadanych
    doc.metadata["doc_id"] = f"doc_{i}"
    doc.metadata["doc_type"] = "markdown"  # Lub inny typ, w zależności od rzeczywistych danych
    doc.metadata["importance"] = "high" if "FAQ" in doc.page_content else "medium"



## 4. Chunkowanie danych

Implementacja różnych strategii chunkowania tekstu:
1. Fixed-size chunking - stałej wielkości
2. Paragraph-based chunking - na podstawie paragrafów
3. Format-specific chunking - bazujące na formatowaniu
4. Semantic chunking - semantyczne (omówimy koncepcję)



In [ ]:

# 4.1 Fixed-size chunking
def fixed_size_chunking(documents, chunk_size=500, chunk_overlap=50):
    """Dzieli dokumenty na chunki o stałej wielkości"""
    text_splitter = CharacterTextSplitter(
        separator="\n\n",
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    
    chunks = []
    for doc in documents:
        doc_chunks = text_splitter.create_documents(
            [doc.page_content], 
            metadatas=[doc.metadata]
        )
        chunks.extend(doc_chunks)
    
    return chunks

# 4.2 Paragraph-based chunking
def paragraph_based_chunking(documents, chunk_size=1000, chunk_overlap=100):
    """Dzieli dokumenty na chunki bazując na paragrafach"""
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ". ", " ", ""],
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    
    chunks = []
    for doc in documents:
        doc_chunks = text_splitter.create_documents(
            [doc.page_content], 
            metadatas=[doc.metadata]
        )
        chunks.extend(doc_chunks)
    
    return chunks

# 4.3 Format-specific chunking (dla Markdown)
def markdown_header_chunking(documents):
    """Dzieli dokumenty Markdown na podstawie nagłówków"""
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=[
            ("#", "header_1"),
            ("##", "header_2"),
            ("###", "header_3"),
        ]
    )
    
    chunks = []
    for doc in documents:
        # Pobierz oryginalne metadane
        original_metadata = doc.metadata
        
        # Podziel tekst według nagłówków Markdown
        md_header_splits = markdown_splitter.split_text(doc.page_content)
        
        # Dodaj oryginalne metadane do każdego chunka
        for chunk in md_header_splits:
            # Połącz metadane z podziału z oryginalnymi metadanymi
            combined_metadata = {**chunk.metadata, **original_metadata}
            chunk.metadata = combined_metadata
            chunks.append(chunk)
    
    return chunks

# 4.4 Semantic chunking (koncepcja)
def semantic_chunking_concept():
    """
    Wyjaśnienie koncepcji semantic chunking.
    
    W rzeczywistej implementacji używalibyśmy modelu embedowania do analizy semantycznej i 
    dzielenia tekstu na podstawie zmian w tematyce.
    
    Możliwe implementacje:
    1. Użycie algorytmów segmentacji tematu
    2. Użycie klastrowania embeddingów zdań
    3. Wykrywanie zmian w wektorach embeddingów dla sąsiednich fragmentów tekstu
    """
    print("""
    Semantic Chunking:
    
    1. Obliczanie embeddingów dla każdego zdania lub paragrafu
    2. Wykrywanie naturalnych granic tematycznych przez analizę odległości wektorów
    3. Grupowanie zdań o podobnej semantyce
    4. Dzielenie na chunki w miejscach, gdzie następuje większa zmiana semantyczna
    
    W praktyce wymaga to:
    - Modelu embeddingów (np. sentence-transformers)
    - Algorytmu segmentacji (np. TextTiling)
    - Technik klastrowania (np. DBSCAN, k-means)
    """)
    
    return None  # W rzeczywistej implementacji zwracalibyśmy chunki



### 🔧 Ćwiczenie 1: Przetestuj różne formy chunkowania


## 5. Generowanie embeddingów

Konwersja chunków tekstu na wektory embeddingów, które będą używane do wyszukiwania podobieństwa semantycznego.



In [ ]:

# Inicjalizacja modelu embeddingów
# W produkcyjnym środowisku można użyć OpenAI, Azure OpenAI lub innych modeli
model_name = "sentence-transformers/all-MiniLM-L6-v2"  # Mały model do celów demonstracyjnych
embedding_model = HuggingFaceEmbeddings(model_name=model_name)

# Generowanie embeddingów dla chunków tekstu
if len(selected_chunks) > 0:
    # Generowanie embeddingów
    chunk_texts = [chunk.page_content for chunk in selected_chunks]
    embeddings = embedding_model.embed_documents(chunk_texts)
    
    print(f"Wygenerowano {len(embeddings)} embeddingów.")
    print(f"Wymiar embeddingu: {len(embeddings[0])}")
    
    # Konwersja na format DataFrame do dalszego przetwarzania
    embedding_rows = []
    for i, (chunk, emb) in enumerate(zip(selected_chunks, embeddings)):
        # Tworzymy wiersz z chunkiem tekstu, metadanymi i embeddingiem
        metadata_str = str(chunk.metadata)  # Konwersja słownika metadanych na string
        embedding_rows.append((i, chunk.page_content, metadata_str, emb))
    
    # Schemat DataFrame
    from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType, FloatType
    
    embedding_schema = StructType([
        StructField("id", IntegerType(), False),
        StructField("content", StringType(), True),
        StructField("metadata", StringType(), True),
        StructField("embedding", ArrayType(FloatType()), True)
    ])
    
    # Tworzenie DataFrame
    embedding_df = spark.createDataFrame(embedding_rows, schema=embedding_schema)
    
    # Zapisanie do tabeli Delta dla dalszego użycia
    embedding_df.write.format("delta").mode("overwrite").saveAsTable("document_embeddings")
    
    print("Zapisano embeddingi do tabeli 'document_embeddings'")
else:
    print("Brak chunków do generowania embeddingów")



## 6. Indeksowanie i przechowywanie

Przygotowanie indeksu wektorowego do efektywnego wyszukiwania podobnych dokumentów.

W Databricks możemy wykorzystać Mosaic AI Vector Search do tego celu.



In [ ]:

# Konfiguracja Mosaic AI Vector Search w Databricks
# W rzeczywistym środowisku należy użyć odpowiednich parametrów dla klastra
try:
    # Sprawdzenie czy tabela z embeddingami istnieje
    embedding_count = spark.table("document_embeddings").count()
    
    if embedding_count > 0:
        # Kreowanie Vector Search Index
        # Uwaga: Poniższy kod jest koncepcyjny i może wymagać dostosowania
        # do rzeczywistej wersji Databricks i dostępnych funkcji
        
        print("""
        Krok koncepcyjny - tworzenie indeksu wektorowego:
        
        W Databricks Mosaic Vector Search utworzylibyśmy indeks:
        
        1. W interfejsie Databricks:
           - Przejdź do zakładki "Catalog"
           - Wybierz "Vector Search" 
           - Utwórz nowy indeks wskazując tabelę "document_embeddings"
           - Wybierz kolumnę "embedding" jako źródło wektorów
           - Skonfiguruj parametry indeksu (np. wymiar, metrykę)
        
        2. Alternatywnie za pomocą kodu:
           ```
           from databricks.vector_search.client import VectorSearchClient
           
           vsc = VectorSearchClient()
           
           vsc.create_index(
               index_name="document_search_index",
               primary_key="id",
               embedding_source={
                   "table_name": "document_embeddings",
                   "embedding_column": "embedding",
                   "dim": len(embeddings[0])
               },
               fields=[{"name": "content", "type": "STRING"}, 
                      {"name": "metadata", "type": "STRING"}]
           )
           ```
        
        3. Po utworzeniu indeksu możemy wykonywać wyszukiwania:
           ```
           query_embedding = embedding_model.embed_query("Jak zresetować hasło?")
           
           results = vsc.query(
               index_name="document_search_index",
               query_vector=query_embedding,
               num_results=5
           )
           ```
        """)
    else:
        print("Brak danych w tabeli 'document_embeddings'")

except Exception as e:
    print(f"Błąd przy dostępie do danych: {e}")



## 7. Testowanie wyszukiwania (symulacja)

Symulacja wyszukiwania w indeksie wektorowym przy użyciu embeddingów.



In [ ]:

# Symulacja wyszukiwania (bez faktycznego indeksu Vector Search)
if 'embedding_model' in locals() and len(selected_chunks) > 0:
    # Zapytanie testowe
    query = "Jak zresetować hasło użytkownika?"
    query_embedding = embedding_model.embed_query(query)
    
    print(f"Zapytanie: '{query}'")
    print(f"Wygenerowano embedding zapytania o wymiarze {len(query_embedding)}")
    
    # Funkcja do obliczania podobieństwa cosinusowego
    def cosine_similarity(vec1, vec2):
        dot_product = np.dot(vec1, vec2)
        norm_vec1 = np.linalg.norm(vec1)
        norm_vec2 = np.linalg.norm(vec2)
        return dot_product / (norm_vec1 * norm_vec2)
    
    # Symulacja wyszukiwania przez obliczenie podobieństwa z każdym chunkiem
    similarities = []
    for i, (chunk, emb) in enumerate(zip(selected_chunks, embeddings)):
        sim_score = cosine_similarity(query_embedding, emb)
        similarities.append((i, chunk, sim_score))
    
    # Sortowanie wyników według podobieństwa (malejąco)
    similarities.sort(key=lambda x: x[2], reverse=True)
    
    # Wyświetlenie najlepszych dopasowań
    print("\nNajlepsze dopasowania:")
    for i, (chunk_id, chunk, score) in enumerate(similarities[:3]):
        print(f"\nWynik {i+1} (Podobieństwo: {score:.4f}):")
        print(f"Treść: {chunk.page_content[:150]}...")
        print(f"Metadane: {chunk.metadata}")
else:
    print("Brak danych do symulacji wyszukiwania")



## 8. Podsumowanie i dalsze kroki

W tym notatniku zaimplementowaliśmy:

1. Preprocessing danych tekstowych
2. Różne strategie chunkowania
3. Generowanie embeddingów
4. Koncepcję indeksowania wektorowego
5. Symulację wyszukiwania semantycznego

W kolejnym ćwiczeniu wykorzystamy te komponenty do budowy pełnego systemu RAG (Retrieval-Augmented Generation) integrującego model językowy.

**Dalsze kroki:**
- Integracja z LLM (np. Azure OpenAI)
- Budowa interfejsu zapytań
- Ewaluacja i optymalizacja wyników
